In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

pd.options.mode.chained_assignment = None

box_DIR = "../data/boxscores_player/"
stats_DIR = "../data/all_in_one_metrics/"

fig_DIR = "../figs/analysis/"

In [ ]:
df = pd.read_csv(stats_DIR + "NBA_LEBRON_FULL.csv")

In [ ]:
df["LEBRON_Mins"] = df["Mins"] * df["LEBRON"]
df["LEBRON_Wins_Mins"] = df["Mins"] * df["LEBRON_WinsAdded"]

In [ ]:
df.columns

In [ ]:
season1 = "2022-23"
df1 = df[df["Season"] == season1]
season2 = "2021-22"
df2 = df[df["Season"] == season2]

In [ ]:
# p = (
#     ggplot() #="LEBRON_WinsAdded"
#     # + geom_density(aes(x="Age", weight = "LEBRON_WinsAdded" ))
#     + geom_density(data = df1,mapping = aes(x="Age", weight="LEBRON_Mins", fill="Season"),alpha=0.5)
#     + geom_density(data = df2,mapping = aes(x="Age", weight="LEBRON_Mins", fill="Season"),alpha=0.5)
#     + scale_fill_manual(values = ["blue","red"])
#     + theme_idv
#     + labs(
#         title = f"NBA Age Distribution {season1} vs {season2}",
#         subtitle = "Weighed by Mins Played and LEBRON",
#         caption = "@sradjoker | source: bball-index.com",
#     )
# )
# p.save(fig_DIR + f"Player Age Distribution {season}.png",dpi=300)
# p

In [ ]:
dfp = pd.read_csv("../data/NBA_players_database.csv")

In [ ]:
dfp.info()

In [ ]:
df["year"] = df["Season"].apply(lambda x: x[:-3]).astype(int)

In [ ]:
dfm = pd.merge(df,dfp,left_on="PLAYER_ID", right_on="pID")

In [ ]:
dfm["Exp"] = dfm["year"] - dfm["From"] +1

In [ ]:
dfm[["PLAYER_NAME","year","From","To","Exp"]]

In [ ]:
dfm.query("Exp>1 & Age == 23 ")

In [ ]:
df1 = dfm.query("Exp>2 & Age == 23 ")
df2 = dfm.query("Exp==1 & Age > 22 ")

In [ ]:
len(df2)

In [ ]:
# fig, ax = plt.subplots(1,1)
# sns.scatterplot(dfm,x="Age",y="Exp")
# plt.show();

In [ ]:
p = (
    ggplot() #="LEBRON_WinsAdded"
    # + geom_density(aes(x="Age", weight = "LEBRON_WinsAdded" ))
    + geom_density(data = df1,mapping = aes(x="LEBRON"),alpha=0.5, fill="red")
    + geom_density(data = df2,mapping = aes(x="LEBRON"),alpha=0.5, fill="blue")
    # + geom_point(data = df2,mapping = aes(x="Age", weight="LEBRON_Mins", fill="Season"),alpha=0.5)
    # + scale_fill_manual(values = ["blue","red"])
    + theme_idv
    + labs(
        title = f"NBA LEBRON Distribution Young Players vs Old Rookies",
        subtitle = "blue: Young Players: Exp>2 and Age 23 | red: Exp = 1 and Age 23 or more",
        caption = "@sradjoker | source: bball-index.com",
    )
)
p.save(fig_DIR + f"Player Age Distribution young vs old.png",dpi=300)
p

In [ ]:
dfa = dfm.query("Exp == 1").groupby("Age")

In [ ]:
dfb = []
keys =  list(dfa.groups)
for key in keys:
    lid = dfa.get_group(key)["PLAYER_ID"].unique()
    dfk = dfm[dfm["PLAYER_ID"].isin(lid)]
    dfke = dfk.groupby("Exp")
    ke = list(dfke.groups)
    for k in ke:
        sums = dfke.get_group(k)[["Mins","LEBRON_Wins_Mins"]].sum()
        lavg = sums.iloc[1]/sums.iloc[0]
        dfkk = pd.DataFrame({"S1_Age": key,"Exp":k,"aLEBRON":lavg},index = [0])
        dfb.append(dfkk)

In [ ]:
dfc = pd.concat(dfb)
dfd = dfc[(dfc["S1_Age"] <25) & (dfc["Exp"] <7) ]

In [ ]:
dfd["S1_Age"] = dfd["S1_Age"].astype("category")

In [ ]:
p = (
    ggplot(data = dfd) #="LEBRON_WinsAdded"
    + geom_point(aes(x="Exp",y="aLEBRON",group="S1_Age",color="S1_Age"))
    + geom_line(aes(x="Exp",y="aLEBRON",group="S1_Age",color="S1_Age"),size=2)
    + scale_color_manual(["red","green","blue","black","cyan","orange"], name ="1st Season Age")
    + theme_idv
    + labs(
        title = f"avg LEBRON vs Exp for Draft Age Groups",
        subtitle = "LEBRON averaged by minutes played",
        caption = "@sradjoker | source: bball-index.com",
    )
)
# p.save(fig_DIR + f"LEBRON vs Exp.png",dpi=300)
p